<a href="https://colab.research.google.com/github/DawidK4/MyNotebooks/blob/main/Predicting_Optimal_Fertilizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e6_path = kagglehub.competition_download('playground-series-s5e6')
dawidkucharski4_optimal_fertilizers_path = kagglehub.dataset_download('dawidkucharski4/optimal-fertilizers')

print('Data source import complete.')


100%|██████████| 11.7M/11.7M [00:00<00:00, 91.9MB/s]

Extracting files...


100%|██████████| 11.7M/11.7M [00:00<00:00, 68.0MB/s]

Extracting files...


Data source import complete.


In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
X = pd.read_csv('train.csv')
y = pd.read_csv('test.csv')
X.head()

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

# EDA

In [ ]:
X.describe()

Data types

In [ ]:
for c in X.columns:
    print(f"Column: {c}, Type: {X[c].dtype}")

Missing values

In [ ]:
nan_counts_per_column = X.isnull().sum()
print("NaN values per column:")
print(nan_counts_per_column)

Value distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numerical_cols = X.select_dtypes(include=['number']).columns

for col in numerical_cols:
    if col != 'id':
        plt.figure(figsize=(10, 6))

        sns.histplot(X[col], kde=True, bins=15, color='skyblue')

        plt.title(f'Distribution of {col}', fontsize=16)
        plt.xlabel(col, fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)

        plt.tight_layout()
        plt.show()

Finding and removing outliers

In [ ]:
def remove_all_outliers_iqr(df):
    """Remove outliers from all numeric columns"""
    df_clean = df.copy()
    numeric_columns = df.select_dtypes(include=['number']).columns

    for column in numeric_columns:
        Q1 = df_clean[column].quantile(0.25)
        Q3 = df_clean[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df_clean = df_clean[
            (df_clean[column] >= lower_bound) &
            (df_clean[column] <= upper_bound)
        ]

    return df_clean

In [ ]:
X_cleaned = remove_all_outliers_iqr(X)
print(f"Original shape: {X.shape}")
print(f"After removing outliers: {X_cleaned.shape}")